# 개요

모델 학습을 위한 코드

- 모델별 가중치 줘서 계산
- 모델 선택 가능

10월 06일 23시



# 1. 파일 업로드 및 기본 세팅

In [ ]:

# AIDU Library Import
from aicentro.session import Session
from aicentro.framework.keras import Keras as AiduFrm
aidu_session = Session(verify=False)
aidu_framework = AiduFrm(session=aidu_session)

# 데이터 파일 경로
data_path = aidu_framework.config.data_dir

import subprocess
import sys

import warnings
# 불필요한 경고 출력을 방지합니다.
warnings.filterwarnings('ignore')


#필요한 모델 다운로드
subprocess.call([sys.executable,"-m","pip","install","xgboost"])
subprocess.call([sys.executable,"-m","pip","install","seaborn"])
subprocess.call([sys.executable,"-m","pip","install","lightgbm"])
subprocess.call([sys.executable,"-m","pip","install","catboost"])




## 러닝 모델 선택

In [ ]:
#러닝을 위한 모델 선택
model_select = {
    
    'randomforest' : 0,
    'lightgbm' : 1,
    'catboost' : 1,
    'xgboost' : 0,
    'softvoting' : 0  

}


## 모델 결과 출력 디렉토리

In [ ]:
#모델 h5 파일 저장 디렉토리
randomforest_model_dir_ = data_path + "/fit/model/randomforest_model.h5"
lightgbm_model_dir = data_path + "/fit/model/lightgbm_model.h5"
catboost_model_dir = data_path + "/fit/model/catboost_model.h5"
xgboost_model_dir = data_path + "/fit/model/xgb_model.h5"
soft_voting_dir = data_path + "/fit/model/soft_voting.h5"

#acc 결과 저장 디렉토리
acc_data_dir = data_path + "/fit/accuracy/acc_data.csv"

#특징 중요도 확인 csv
feature_importance_dir = data_path + "/fit/feature_importance/model_Feature_Importance.csv"

# 최종 결과 파일명
result_file_name = data_path +'/제출데이터/2차제출_1007_0100.csv'

# 데이터 불러오기 및 기본 처리


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd


#Train file 데이터 디렉토리
train_file_dir = data_path + "/episode1_train.csv"
train_ob_file_dir = data_path + "/episode1_train_ob.csv"

#Test file 데이터 디렉토리
test_file_dir = data_path + "/episode1_test.csv"
test_ob_file_dir = data_path + "/episode1_test_ob.csv"


#전처리 후 test 파일명
test_file_name =data_path + "/pre_data/test_preprocessing.csv"

#전처리 후 train 파일명
train_file_name = data_path + "/pre_data/train_preprocessing.csv"


train_df = pd.read_csv(train_file_name)
test_df = pd.read_csv(test_file_name)



In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
def model_split(input_split_df, col_name ='include_ob' ):
    '''
    ob 데이터가 있는 것과 ,아닌것으로 나누는 함수
    
    param 
    input_split_df : 분리하고 싶은 데이터
    col_name : 분리를 하기위한 구분 컬럼
    '''
    
    #데이터 2개 선언
    include_ob_df = pd.DataFrame()
    include_not_ob_df = pd.DataFrame()
    
    #구분 컬럼이 1 / 0 에 따라 판단하여 데이터 분리
    include_ob_df = input_split_df[input_split_df[col_name]==1.0]
    include_not_ob_df = input_split_df[input_split_df[col_name]==0.0]
    
    
    include_ob_df.drop(['include_ob'], axis=1, inplace =True)
    include_not_ob_df.drop(['tm_athrt_trmn_cnt', 'tm_cant_call_cnt', 
       'tm_claim_cnt', 'tm_etc_cnt', 'tm_handling_complaint_cnt',
       'tm_npay_cnt', 'tm_promise_pay_cnt', 'tm_term_stop_cnt',
       'tm_use_stop_cnt', 'sm_athrt_trmn_cnt', 'sm_npay_cnt',
        'sm_thismonth_cnt', 'sm_use_stop_cnt',
       'include_ob'],axis=1, inplace=True)
    
    return include_ob_df, include_not_ob_df

    
    

In [ ]:
#ob 데이터가 있는것과 아닌것으로 분류
train_ob_df , train_not_ob_df =model_split(train_df,'include_ob')
test_ob_df , test_not_ob_df =model_split(test_df,'include_ob')

#뒤에 결과값 출력을 위해 id와 include_ob 저장
temp_df = pd.read_csv(test_file_dir)
test_id_df = pd.DataFrame()
test_id_df['id'] = temp_df['id']
test_id_df['include_ob'] = test_df['include_ob']

In [ ]:
test_not_ob_df.shape

In [ ]:

#데이터도 include_ob 컬럼 지움
train_df.drop(['include_ob'],axis=1, inplace=True)
test_df.drop(['include_ob'],axis=1, inplace=True)

print(train_ob_df.shape, train_not_ob_df.shape)
print(test_ob_df.shape, test_not_ob_df.shape)


In [ ]:
#학습하기위한 train과 validation 나눔
# 일반 데이터
x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'],random_state=21)

# ob데이터가 있는것고 없는것으로 나눈것
x_train_ob, x_valid_ob, y_train_ob, y_valid_ob = train_test_split(train_ob_df.drop(['label_payment_yn'],axis=1), train_ob_df['label_payment_yn'],random_state=21)
x_train_not_ob, x_valid_not_ob, y_train_not_ob, y_valid_not_ob = train_test_split(train_not_ob_df.drop(['label_payment_yn'],axis=1), train_not_ob_df['label_payment_yn'],random_state=21)


### 모델 학습 후 결과 시각화

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

my_predictions = {}

colors = ['r', 'c', 'm', 'y', 'k', 'khaki', 'teal', 'orchid', 'sandybrown',
          'greenyellow', 'dodgerblue', 'deepskyblue', 'rosybrown', 'firebrick',
          'deeppink', 'crimson', 'salmon', 'darkred', 'olivedrab', 'olive', 
          'forestgreen', 'royalblue', 'indigo', 'navy', 'mediumpurple', 'chocolate',
          'gold', 'darkorange', 'seagreen', 'turquoise', 'steelblue', 'slategray', 
          'peru', 'midnightblue', 'slateblue', 'dimgray', 'cadetblue', 'tomato'
         ]


#acc를 구해서 시각화해줌
def acc_eval(name_, pred, actual):
    global predictions
    global colors

    acc = (pred==actual).mean()
    my_predictions[name_] = acc


    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=False)  # 정확도 내림차순으로 sort
    
    df = pd.DataFrame(y_value, columns=['model', 'acc'])
    print(df)
    min_ = df['acc'].min() -1
    max_ = 1.2
    
    length = len(df)
    
    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df['acc'])
    
    for i, v in enumerate(df['acc']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v+0.1, i, str(round(v, 3)), color='k', fontsize=15, fontweight='bold')
        
    plt.title('Accuracy', fontsize=18)
    plt.xlim(min_,max_)
    
    plt.show()
    
#acc를 구해서 시각화해줌(cross_val_score 사용하는 경우)
def acc_eval_add(name_, acc):
    global predictions
    global colors

    my_predictions[name_] = acc


    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=False)  # 정확도 내림차순으로 sort
    
    df = pd.DataFrame(y_value, columns=['model', 'acc'])
    print(df)
    min_ = df['acc'].min() -1
    max_ = 1.2
    
    length = len(df)
    
    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df['acc'])
    
    for i, v in enumerate(df['acc']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v+0.1, i, str(round(v, 3)), color='k', fontsize=15, fontweight='bold')
        
    plt.title('Accuracy', fontsize=18)
    plt.xlim(min_,max_)
    
    plt.show()
    
    df.to_csv(acc_data_dir,index=False)
    

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
import time
from sklearn.model_selection import KFold

# 3. 모델학습 (랜덤 포레스트, XGboost, Light GBM, CATboost )

### 3.1. 랜덤포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def randomforest_model_fit (x_train_random, x_valid_random, y_train_random, y_valid_random, model_name='RF'):
    '''
    랜덤포레스트 모델 학습을 위한 함수
    
    param
    x_train_cat: x train data
    x_valid_cat: x valid data
    y_train_cat: y train data
    y_valid_cat: y valid data
    model_name: 모델 학습 후 그래프 출력을 위한 텍스트
    '''
    
    #모델선언
    model_rf = RandomForestClassifier(n_jobs=-1)

    score = cross_val_score(model_rf, x_train_random, y_train_random).mean()
    cross_val_score(model_rf, x_valid_random, y_valid_random).mean()
    
    #학습
    model_rf.fit(x_train_random, y_train_random)
    
    #예측 및 결과 확인
    pred=model_rf.predict(x_valid_random)
    acc_eval_add(model_name,score) #기본
    
    #모델 및 예측값 return
    return model_rf, pred
    

In [ ]:
#랜덤포레스트 모델 러닝 선택시
   
if model_select['randomforest'] == 1:
    randomforest_model, randomforest_pred = randomforest_model_fit (x_train, x_valid, y_train, y_valid, 'randomforest' )
    

### 3.2. XGBoost 

In [ ]:
from xgboost import XGBClassifier

def xgboost_model_fit (x_train_xg, x_valid_xg, y_train_xg, y_valid_xg, model_name='XGB'):
    '''
    Cat boost 모델 학습을 위한 함수
    
    param
    x_train_cat: x train data
    x_valid_cat: x valid data
    y_train_cat: y train data
    y_valid_cat: y valid data
    model_name: 모델 학습 후 그래프 출력을 위한 텍스트
    '''
    
     #모델 선언
    xgb_model = XGBClassifier(n_jobs=-1)

    start=time.time()

    eval_set = [(x_valid_xg, y_valid_xg)]
    
    #학습
    #logloss가 10회 계선되지 않으면 중단하도록 early stopping 설정 n_estimators=100에서는 얼리스탑하지 않음
    xgb_model.fit(x_train_xg, y_train_xg, early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=2)

    (time.time()-start)/60
    
    pred= xgb_model.predict(x_valid_xg)
    
    acc_eval(model_name, pred,y_valid) #기본
    
    #모델 및 예측값 return
    return xgb_model, pred

In [ ]:
#xgboost 모델 선택시

if model_select['xgboost'] == 1:
    #모델 학습
    xgboost_model, xgboost_pred = xgboost_model_fit (x_train, x_valid, y_train, y_valid, 'xgboost' )
    #모델 save
    xgboost_model.save_model(xgboost_model_dir )
    


### 3.3 CATBoost

In [ ]:
from catboost import CatBoostClassifier

def catboost_model_fit (x_train_cat, x_valid_cat, y_train_cat, y_valid_cat, model_name='CAT'):
    '''
    Cat boost 모델 학습을 위한 함수
    
    param
    x_train_cat: x train data
    x_valid_cat: x valid data
    y_train_cat: y train data
    y_valid_cat: y valid data
    model_name: 모델 학습 후 그래프 출력을 위한 텍스트
    '''
    #모델 선언
    cat_model= CatBoostClassifier()
    start=time.time()
    
    #학습
    cat_model.fit(x_train_cat, y_train_cat, verbose=2 )
    
    #예측 및 결과 확인
    pred = cat_model.predict(x_valid_cat)
    acc_eval(model_name ,pred, y_valid_cat)
    
    #모델 및 예측값 return
    return cat_model, pred




In [ ]:
def catboost_model_kfold_fit (x_train_cat, x_valid_cat, y_train_cat, y_valid_cat, model_name='CAT'):
    '''
    Cat boost 모델 학습을 위한 함수 , KFOLD 포함
    
    param
    x_train_cat: x train data
    x_valid_cat: x valid data
    y_train_cat: y train data
    y_valid_cat: y valid data
    model_name: 모델 학습 후 그래프 출력을 위한 텍스트
    '''
    
    #모델 선언
    cat_model= CatBoostClassifier()
    start=time.time()
    
    kf = KFold(n_splits=2, shuffle=True, random_state = 71)
    for tr_idx, va_idx in kf.split(x_train):
        x_train_fold, x_valid_fold = x_train_cat.iloc[tr_idx], x_train_cat.iloc[va_idx]
        y_train_fold, y_valid_fold = y_train_cat.iloc[tr_idx], y_train_cat.iloc[va_idx]

        cat_model.fit(x_train_fold, y_train_fold, verbose=0 )

    #예측 및 결과 확인
    pred = cat_model.predict(x_valid_cat)
    acc_eval(model_name ,pred, y_valid_cat)
    
    #모델 및 예측값 return
    return cat_model, pred



In [ ]:

if model_select['catboost'] == 1:
    #모델 학습 진행
#     cat_ob_model,cat_ob_pred = cat_boost_model (x_train_ob, x_valid_ob, y_train_ob, y_valid_ob, 'CAT_OB' )
#     cat_not_ob_model,cat_not_ob_pred = cat_boost_model (x_train_not_ob, x_valid_not_ob, y_train_not_ob, y_valid_not_ob, 'CAT_NOT_OB' )
    
    catboost_model, catboost_pred = catboost_model_fit (x_train, x_valid, y_train, y_valid, 'catboost' )
    catboost_model.save_model(catboost_model_dir)
    

### 3.4 LightGBM

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
def lightgbm_model_fit(x_train_lgbm, x_valid_lgbm, y_train_lgbm, y_valid_lgbm, model_name='LGBM'):

    '''
    lgbm 모델 학습을 위한 함수 
    
    param
    x_train_cat: x train data
    x_valid_cat: x valid data
    y_train_cat: y train data
    y_valid_cat: y valid data
    model_name: 모델 학습 후 그래프 출력을 위한 텍스트
    '''
    
    lgbm_model = LGBMClassifier()

    
    start=time.time()

    eval_set = [(x_valid_lgbm, y_valid_lgbm)]

    #logloss가 10회 계선되지 않으면 중단하도록 early stopping 설정 n_estimators=100에서는 얼리스탑하지 않음
    lgbm_model.fit(x_train_lgbm, y_train_lgbm, early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=2)

    (time.time()-start)/60

    pred = lgbm_model.predict(x_valid_lgbm)
    
    acc_eval(model_name, pred,y_valid_lgbm) #기본
    
    return lgbm_model, pred

In [ ]:
# KFold 로 교차 검증 실시

def lightgbm_model_kfold_fit (x_train_lgbm, x_valid_lgbm, y_train_lgbm, y_valid_lgbm, model_name='LGBM'):

    lgbm_model = LGBMClassifier()

    kf = KFold(n_splits=4, shuffle=True, random_state = 71)
    for tr_idx, va_idx in kf.split(x_train):
        x_train_fold, x_valid_fold = x_train_lgbm.iloc[tr_idx], x_train_lgbm.iloc[va_idx]
        y_train_fold, y_valid_fold = y_train_lgbm.iloc[tr_idx], y_train_lgbm.iloc[va_idx]

        start=time.time()

        eval_set = [(x_valid_fold, y_valid_fold)]

        #logloss가 10회 계선되지 않으면 중단하도록 early stopping 설정 n_estimators=100에서는 얼리스탑하지 않음
        lgbm_model.fit(x_train_fold, y_train_fold, early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=2)

        (time.time()-start)/60
        
    
    pred = lgbm_model.predict(x_valid_lgbm)
    
    acc_eval(model_name, pred, y_valid_lgbm) #기본
    
    return lgbm_model, pred

In [ ]:
#모델 학습 시작
if model_select['lightgbm'] ==1 :
    
    #일반 모델 
    lightgbm_model, lightgbm_pred = lightgbm_model_fit (x_train, x_valid, y_train, y_valid, 'lightgbm' )
    lightgbm_model.booster_.save_model(lightgbm_model_dir )
    
    #OB 분리 모델
#     lgbm_ob_model ,lgbm_ob_pred = lgbm_model(x_train_ob, x_valid_ob, y_train_ob, y_valid_ob, 'LGBM_OB')
#     lgbm_not_ob_model ,lgbm_not_ob_pred = lgbm_model(x_train_not_ob, x_valid_not_ob, y_train_not_ob, y_valid_not_ob , 'LGBM_not_OB')

#     lgbm_ob_model.booster_.save_model(data_path + "/fit/LGBM_ob_model.h5" )
#     lgbm_not_ob_model.booster_.save_model(data_path + "/fit/LGBM_not_ob_model.h5" )

# 4. 모델 선택 및 앙상블
## softvoting

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
# VotingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import cross_validate

# 라이브러리 import
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score





In [ ]:

def softvoting_model_fit (x_train_soft, x_valid_soft, y_train_soft, y_valid_soft, model_name='softvote'):
    '''
    softvote 모델 학습을 위한 함수 , KFOLD 포함
    
    param
    x_train_cat: x train data
    x_valid_cat: x valid data
    y_train_cat: y train data
    y_valid_cat: y valid data
    model_name: 모델 학습 후 그래프 출력을 위한 텍스트
    '''
    
    # ensemble 할 model 정의
    models = [
        #('rfc', RandomForestClassifier()),
        ('xgb', XGBClassifier()),
        ('lgbm', LGBMClassifier()),
        ('dtc', DecisionTreeClassifier()),

    ]

    # soft vote
    soft_vote  = VotingClassifier(models, voting='soft')
    #soft_vote_cv = cross_validate(soft_vote, x_train_soft, y_train_soft, cv=2)
    soft_vote.fit(x_train_soft, y_train_soft)
    pred = soft_vote.predict(x_valid_soft)
    
    acc_eval(model_name, pred, y_valid_soft) #기본

    # print
    print("Soft_Vote Test Accuracy: ", accuracy_score(y_valid_soft, pred))
    print(classification_report(y_valid, pred, target_names=['N','Y']))
    print(confusion_matrix(y_true=y_valid, y_pred=pred))

    return soft_vote, pred


In [ ]:
#모델학습 
if model_select['softvoting'] == 1:
    softvoting_model, soft_pred = softvoting_model_fit(x_train, x_valid, y_train, y_valid, 'softvoting' )
    softvoting_model.save_model(soft_voting_model_dir )

## 변수 중요도 출력

In [ ]:
'''
    변수 중요도를 출력
    frature_importance_model 에 학습한 모델 하나 선택
    'randomforest' , 'xgboost', 'catboost',  'lightgbm', 'softvoting'    
'''

feature_importance_model = "catboost"

#모델 실수로 선택 안했거나 잘못 골랐을때 하나 골라서 출력
if (feature_importance_model ==" ")  or  (model_select[feature_importance_model] != 1):
    for col in model_select:
        if model_select[col] == 1 :
            feature_importance_model = col
           
# 해당 모델 가져오기
if feature_importance_model == 'randomforest':
    fit_model = randomforest_model
elif feature_importance_model == 'lightgbm':
    fit_model = lightgbm_model
elif feature_importance_model == 'catboost':
    fit_model = catboost_model
elif feature_importance_model == 'xgboost':
    fit_model = xgboost_model
elif feature_importance_model == 'softvoting':
    fit_model = soft_pred

feature_importance_df = pd.DataFrame()

#컬럼명과 중요도 데이터 가져옴
feature_importance_df['col'] = x_train.columns
feature_importance_df['val'] = fit_model.feature_importances_

#중요 순서로 sort
feature_importance_df.sort_values(by='val',axis=0,ascending=False,inplace=True)

#그래프 출력
plt.figure(figsize=(25, 5))
plt.bar(feature_importance_df['col'],feature_importance_df['val'])
plt.xticks(rotation=90)
plt.show()

#파일 출력
feature_importance_df.to_csv( feature_importance_dir )



# 모델 출력

## 1. 모델 여러개 합쳐서 결과 계산하기

In [ ]:
#모델 가중치
model_weight = {
    
    'randomforest' : 0.1,
    'xgboost' :0.20,
    'catboost' :0.30,
    'lightgbm' : 0.4,
    'softvoting' : 0  
    
}

#모델 학습 선택 출력
model_select

### 합친 모델로 valid accuracy 계산

In [ ]:

valid_accuracy_df = pd.DataFrame()

valid_accuracy_df['result']= (list(np.zeros(y_valid.shape[0])))

for index in model_weight:
    
    #모델을 학습 했으면
    if model_select[index] == 1:
        
        # 해당 모델 가져오기
        if index == 'randomforest':
            index_pred = randomforest_pred
        elif index == 'lightgbm':
            index_pred = lightgbm_pred
        elif index == 'catboost':
            index_pred = catboost_pred
        elif index == 'xgboost':
            index_pred = xgboost_pred
        elif index == 'softvoting':
            index_pred = soft_pred
        
        #각 모델별로 가중치 만듬 반영하여 계산
        valid_accuracy_df['result'] =  valid_accuracy_df['result'] +pd.Series(index_pred)*model_weight[index]
 
valid_accuracy_df
valid_accuracy_df['result'].value_counts()

In [ ]:
#가중치로 계산한 값을 확인
valid_accuracy_df.loc[valid_accuracy_df['result']>=0.5, 'result'] = 1
valid_accuracy_df.loc[valid_accuracy_df['result']<0.5, 'result'] = 0

valid_accuracy_df['result'].astype('int')



acc_eval('model_weight_cal', valid_accuracy_df['result'].values, y_valid ) #기본


### test 결과 출력

In [ ]:

model_result_df = pd.DataFrame()
model_result_df['result'] = (list(np.zeros(test_df.shape[0])))

#학습한 모델만 각각 예측 결과 확인하여 데이터 프레임 만듬
for index in model_select:
    
    #모델을 학습 했으면
    if model_select[index] == 1:
        
        # 해당 모델 가져오기
        if index == 'randomforest':
            index_model = randomforest_model
        elif index == 'lightgbm':
            index_model = lightgbm_model
        elif index == 'catboost':
            index_model = catboost_model
        elif index == 'xgboost':
            index_model = xgboost_model
        elif index == 'softvoting':
            index_model = softvoting_model
        
        #모델 예측
        index_pred = index_model.predict(test_df)
        
        
        #예측값 데이터프레임에 추가
        model_result_df[index] = pd.Series(index_pred)
        
        
        #각 모델별로 가중치 만듬 반영하여 계산
        model_result_df['result'] =  model_result_df['result'] +pd.Series(index_pred)*model_weight[index]
        
        
        #결과 출력
        print(index ,"예측 완료")




In [ ]:
model_result_df
model_result_df['result'].value_counts()

In [ ]:
model_result_df

In [ ]:
#계산한 weight를 기준으로 0.5보다 크면 1 / O으로 대치

model_result_df['result'].astype('float')


model_result_df.loc[model_result_df['result']>=0.5, 'pred']='Y'
model_result_df.loc[model_result_df['result']<0.5, 'pred']='N'

model_result_df
    

In [ ]:
#ID값만 다시 출력하기 위해 불러옴
test_raw_df = pd.read_csv(test_file_dir)

result = pd.concat([test_raw_df['id'],pd.Series(model_result_df['result'])], axis=1)

result.to_csv(result_file_name, header=False, index=False, encoding='utf-8')
print("결과 예측 파일이 출력되었습니다. : ", result_file_name)
result

## 2. 일반 모델 결과 출력

In [ ]:

# if randomforest_model == True:
#     result_model = rf_model
# elif lightgbm_model == True:
#     result_model = lgbm_model
# elif catboost_model == True:
#     result_model = cat_model
# elif xgboost_model == True:
#     result_model = xgb_model
# elif soft_voting_model == True:
#     result_model = soft_model


In [ ]:
# #결과 출력할 모델 선택
# result_pred = result_model.predict(test_df)

# #ID값만 다시 출력하기 위해 불러옴
# test_raw_df = pd.read_csv(test_file_dir)

# result = pd.concat([test_raw_df['id'],pd.Series(result_pred)], axis=1)

# result.loc[result[0]==1, 0]='Y'
# result.loc[result[0]==0, 0]='N'

# result.to_csv(result_file_name, header=False, index=False, encoding='utf-8')
